# **Введение**

В этом модуле мы поговорим об ещё одном направлении разведывательного анализа данных — проектировании признаков (Feature Engineering). Этот процесс также называют конструированием признаков или разработкой признаков. Все эти термины описывают процесс создания признаков, которые нужны для обучения модели.

Но мы же уже собрали данные на этапе сбора данных и подготовили данные на этапе очистки… Для чего повторно создавать части в EDA?

**[Пример: При прогнозировании дефолта клиента банка для создания прогнозной модели очень важно иметь информацию о дате выдачи кредита, плановой и действительной датах погашения. Эти даты дают нам представление о характере погашения кредита: был он выплачен вовремя или раньше срока, с просрочкой. ]**

В отличие от человека, модель машинного обучения не может проанализировать эти даты. А вот информация о просрочке (разница между плановой и фактической датой погашения) представлена в числовом формате и может сообщить некую информацию модели. Число, равное или меньше 0, говорит о клиенте как о благонадёжном заёмщике; число больше нуля характеризует заёмщика как должника. Такое конструирование признаков помогает дата-сайентисту не утратить часть данных, а также представить их в более «выгодном» свете.

На этапе проектирования признаков специалисты по данным преследуют такие же цели, как на этапе сбора данных — сконструировать как можно больше признаков, чтобы далее отобрать из них лучшие.

In [2]:
import pandas as pd
from IPython.display import display
import datetime as dt
data = pd.read_csv('data/wine_cleared.csv')

***
## **Создание признаков**

*Вы уже знакомы с методами создания столбцов в pandas из модуля PYTHON-11. Базовые приемы работы с данными в Pandas. Юнит 2. Базовые операции со столбцами DataFrame. Советуем повторить этот материал.*

Новые признаки могут быть сконструированы двумя способами: 

* с помощью внешних источников данных;
* из существующего набора данных.

**[В колл-центрах часто используются системы автоматического подбора номеров для дозвона (рекомендация). Если клиент во время прошлого созвона сбросил трубку, то автоматическая система должна снижать рекомендацию этого номера. Клиент вряд ли ответит после того, как бросил трубку.]**

В выделении этого факта нам поможет создание нового признака «сброс трубки» из информации в существующем датасете. Эта информация может быть получена из уже существующих признаков, например если время звонка составило меньше 10 секунд.

**[Для определения дефолтности клиента банки делают запрос в БКИ (Бюро Кредитных Историй (БКИ) занимается организацией, хранением и предоставлением кредитных отчётов, в которых содержатся сведения о принятых на себя обязательствах по договорам займа и кредита) с целью получения дополнительной информации по клиенту. Такая информация о невыплаченных долгах другим организациям может быть решающим фактором в вынесении решения по кредиту. Эту информацию можно получить с использованием внешних источников данных.]**

Такая информация о невыплаченных долгах другим организациям может быть решающим фактором в вынесении решения по кредиту. Эту информацию можно получить с использованием внешних источников данных.

Новые признаки можно сконструировать из уже существующего набора данных несколькими способами: 

1. разбор категорий; 
2. разбор даты и времени;
3. разбор числовых признаков;
4. разбор текста.

***
#### **РАЗБОР ДАТЫ И ВРЕМЕНИ**

Например, в нашем датасете в названии вина можно увидеть год производства вина. Этот признак может оказаться важным для определения рейтинга вина. Однако, пока он заключен в названии вина, модель не сможет его использовать.  
![год](https://lms.skillfactory.ru/assets/courseware/v1/c9a5adff4dea2b2460098cc3672dfbde/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-eda-3-1.png)

Можно выделить год, месяц и часы в отдельные признаки: время суток, времена года и так далее. И если вы считаете, что ваш процесс зависит от времени, посвятите этому этапу как можно больше усилий.
***
#### **РАЗБОР ЧИСЛОВЫХ ВЕЛИЧИН**

В наборе данных очень часто встречаются числовые величины. Это могут быть вес, рост, расстояние и так далее. Вы можете преобразовывать их, создавать новые признаки, которые могут оказаться лучше исходных.

Например, в датасете винных обзоров числовым признаком является цена за бутылку вина. Вы можете округлить цену 35.363389$ в 35$, избавив данные от лишней информации.

✍️ **Создадим новый признак price_round, означающий округлённую до целого числа цену за бутылку вина:**

In [3]:
# для удобства сразу преобразуем признак в int
data['price_round'] = data['price'].round().astype(int)

![новый признак](https://lms.skillfactory.ru/assets/courseware/v1/c01a5b05d91c845384fbecfb53005657/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-eda-3-32.png)

Для таких преобразований важен контекст. В контексте вина для сомелье цена в 35$ и 35.363389$ одинакова. Вы делаете округление признака, чтобы модель также не сосредотачивалась на сотых. Это может помочь улучшить предсказание.

**[Однако такое преобразование неуместно в ситуациях, когда сотые важны. Например, при предсказании курса лиры стоимость валюты в 6.12 руб. и в 6.84 руб. — абсолютно разные ситуации.]**

Помимо округления для создания новых признаков могут применяться такие популярные операции, как логарифмирование числового признака или извлечение его корня.

***
#### **РАЗБОР ТЕКСТА**

Текстовые признаки могут содержать в себе большое количество информации. Например, если бы наша модель умела распознавать текст, то из описания вина мы бы могли понять, какими словами сомелье чаще характеризуют качественное вино. Вы можете воспользоваться методами извлечения слов из текста, чтобы получить новые признаки.

В названии вина вы можете заметить указание года произведения вина. Чтобы узнать, влияет ли год на рейтинг вина, вам необходимо выделить год в отдельный признак.

Для того, чтобы извлечь из строки число, обозначающее год, необходимо воспользоваться **регулярными выражениями**.

**Регулярные выражения (regexp, или regex)** — это механизм для поиска и замены текста. Это шаблоны, которые используются для поиска соответствующей части текста.

**[Например, с помощью такого регулярного выражения [^@ \t\r\n]+@[^@ \t\r\n]+\.[^@ \t\r\n]+ можно найти любой email в тексте.]**

![email в тексте](https://lms.skillfactory.ru/assets/courseware/v1/37696fcdaef05494ffefda4c7e4b5a44/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst-eda-3-33.png)

Синтаксис регулярных выражений необычен и содержит различные символы, знаки препинания, математических операций и так далее. Подробнее ознакомиться с синтаксисом регулярных выражений поможет статья [Python RegEx: практическое применение регулярок](https://tproger.ru/translations/regular-expression-python/).

Реализация такого механизма существует в pandas в работе со строками. Для того чтобы найти все числа в каждом значении серии, воспользуемся методом [str.findall()](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.findall.html). Метод возвращает все совпадения с заданным шаблоном в серии pandas. 


In [4]:
regex = '\d{4}' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)
display(data.head(2))

,Unnamed: 0,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,price_round,year
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,35,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,15,2011


Разберём регулярное выражение **\d**:

* **\d** — класс символов, обозначает соответствие цифрам в диапазоне цифр [0-9];
* **{4}** в шаблоне означает искать четыре вхождения символа, указанного ранее. В нашем случае это будут четырехзначные числа.

Таким образом, **\d{4}** означает поиск четырехзначных чисел в заданной строке.  
Проверить, насколько точно работает ваше регулярное выражение, а также найти реализацию популярных регулярных выражений (например, поиска номера телефона), вы сможете на сайте **https://ihateregex.io**.

Однако при поиске числа методом **data['title'].str.findall(regex)** результатом выполнения является список найденных цифр. Поэтому необходимо извлечь первый элемент из списка найденных методом **str.get(0)**, где 0 — первый элемент в списке найденных чисел.

Используя методы работы со строками pandas и регулярные выражения, можно извлечь любую информацию для новых признаков: даты, слова и выражения.

***
#### **РАЗБОР КАТЕГОРИЙ**

В наборе данных винных обзоров самая популярная страна-производитель вина — США. Возможно, это не случайность, и факт производства в США влияет на рейтинг вина. Выделим этот факт.

Вы можете создать новый бинарный признак is_usa и присвоить ему 1 в случае, если вино произведено в США, иначе — 0.


In [5]:
data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)

In [6]:
# Выберите из списка две самых популярных (помимо США) страны, производящих вино.
data['country'].value_counts().head()

US          54504
France      22093
Italy       19540
Spain        6645
Portugal     5691
Name: country, dtype: int64

In [7]:
# Создайте бинарные признаки is_france, is_italy наподобие признака is_usa
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)
data['is_france'].sum()
data['is_italy'].sum()

19540

In [8]:
# Создайте новый бинарный признак old_wine, где значение 1 будет обозначать, что вино старше 2010 года.
data['year'] = pd.to_datetime(data['year'], errors='coerce')
# для сравнения используем год, заполняем значения признака old_wine, где год вина меньше 2010
data['old_wine'] = data['year'].apply(lambda x: 1 if x.year < 2010 else 0) 
data['old_wine'].sum()

39765

In [25]:
# Создайте новый признак locality из признака title, который будет обозначать название долины/местности производства вина.
regex = '\((.*?)\)' # регулярное выражение для нахождения ()
data['locality'] = data['title'].str.findall(regex).str.get(0)
display(data['locality'].value_counts())

Napa Valley                          4478
Columbia Valley (WA                  4124
Russian River Valley                 3091
California                           2629
Paso Robles                          2350
                                     ... 
Finca Élez                              1
Valpolicella Valpantena Superiore       1
Moscato di Pantelleria                  1
Paardeberg                              1
Paestum                                 1
Name: locality, Length: 1562, dtype: int64